In [6]:
import re, json
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
def scrape_vinatis(page:int):
    # On récupère le site de Vinatis
    vin = requests.get(f"https://www.vinatis.com/achat-vin?page={page}").text
    # Beautiful soup est une lib pour scraper les sites web
    soup = BeautifulSoup(vin, "html.parser")
    # On récupère a balise <script> qui contient les infos de vin 
    scripts = soup.find_all("script")
    # Pour chaque balise <script>
    for script in scripts:
    # Tous nos vins sont dans une variable javascript appelée product_elastic
        if script.string and "product_elastic" in script.string:
            # On crée une regex pour extraire le JSON de cette variable
            match = re.search(r"var\s+product_elastic\s*=\s*({.*?});", script.string, re.DOTALL)
            if match:
                # On charge le JSON en dictionnaire et on le retourne sous forme de dataframe
                data = json.loads(match.group(1))
                df = pd.DataFrame(data["products"])
                return df
            
           

In [10]:
all_dfs = []

for page in range(1, 151):
    df = scrape_vinatis(page)
    if df is not None:
        all_dfs.append(df)

df_final = pd.concat(all_dfs, ignore_index=True)
df_final.to_csv("vins_vinatis_150_pages.csv", index=False)